In [1]:
from pdfquery import PDFQuery
import pdfminer
from pdfminer.pdfpage import PDFPage, PDFTextExtractionNotAllowed
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.layout import LAParams
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LTTextBoxHorizontal, LTRect, LTLine


import matplotlib.pyplot as plt
from matplotlib import patches
%matplotlib inline

import pandas as pd

In [3]:
def extract_page_layouts(file):
    """
    Extracts LTPage objects from a pdf file.
    modified from: http://www.degeneratestate.org/posts/2016/Jun/15/extracting-tabular-data-from-pdfs/
    Tests show that using PDFQuery to extract the document is ~ 5 times faster than pdfminer.
    """
    laparams = LAParams()
    
    with open(file, mode='rb') as pdf_file:
        print("Open document %s" % pdf_file.name)
        document = PDFQuery(pdf_file).doc

        if not document.is_extractable:
            raise PDFTextExtractionNotAllowed

        rsrcmgr = PDFResourceManager()
        device = PDFPageAggregator(rsrcmgr, laparams=laparams)
        interpreter = PDFPageInterpreter(rsrcmgr, device)

        layouts = []
        for page in PDFPage.create_pages(document):
            interpreter.process_page(page)
            layouts.append(device.get_result())
    
    return layouts

In [5]:
TEXT_ELEMENTS = [
    pdfminer.layout.LTTextBox,
    pdfminer.layout.LTTextBoxHorizontal,
    pdfminer.layout.LTTextLine,
    pdfminer.layout.LTTextLineHorizontal
]

FILE_NAME = "test.pdf"
COL_MARGIN = 0.5
page_layouts = extract_page_layouts(FILE_NAME)
print(pages)
print("Number of pages: %d" % len(page_layouts))



Open document test.pdf
[<LTPage(1) 0.000,0.000,792.000,612.000 rotate=0>, <LTPage(2) 0.000,0.000,792.000,612.000 rotate=0>, <LTPage(3) 0.000,0.000,792.000,612.000 rotate=0>]
Number of pages: 3


In [6]:
def extract_single_page_text(current_page):
    text = []
    for elem in current_page:
        if isinstance(elem, pdfminer.layout.LTTextBoxHorizontal):
            text.append(elem)
    return text

In [ ]:
raw_text = extract_single_page_text(page_layouts[0])
raw_text

In [15]:
def flatten(lst):
    """Flattens a list of lists"""
    return [item for sublist in lst for item in sublist]

def extract_characters(element):
    """
    Recursively extracts individual characters from 
    text elements. 
    """
    if isinstance(element, pdfminer.layout.LTChar):
        return [element]

    if any(isinstance(element, i) for i in TEXT_ELEMENTS):
        return flatten([extract_characters(e) for e in element])

    if isinstance(element, list):
        return flatten([extract_characters(l) for l in element])

    return []

In [16]:
raw_characters = extract_characters(raw_text)
raw_characters

[<LTChar 39.600,563.532,45.099,572.532 matrix=[9.00,0.00,0.00,9.00, (39.60,565.44)] font='AAAAAC+Arial-BoldMT' adv=0.611 text='T'>,
 <LTChar 45.099,563.532,51.597,572.532 matrix=[9.00,0.00,0.00,9.00, (45.10,565.44)] font='AAAAAC+Arial-BoldMT' adv=0.722 text='D'>,
 <LTChar 51.597,563.532,54.099,572.532 matrix=[9.00,0.00,0.00,9.00, (51.60,565.44)] font='AAAAAC+Arial-BoldMT' adv=0.278 text=' '>,
 <LTChar 54.099,563.532,60.597,572.532 matrix=[9.00,0.00,0.00,9.00, (54.10,565.44)] font='AAAAAC+Arial-BoldMT' adv=0.722 text='A'>,
 <LTChar 60.597,563.532,68.598,572.532 matrix=[9.00,0.00,0.00,9.00, (60.60,565.44)] font='AAAAAC+Arial-BoldMT' adv=0.889 text='m'>,
 <LTChar 68.598,563.532,73.602,572.532 matrix=[9.00,0.00,0.00,9.00, (68.60,565.44)] font='AAAAAC+Arial-BoldMT' adv=0.556 text='e'>,
 <LTChar 73.602,563.532,77.103,572.532 matrix=[9.00,0.00,0.00,9.00, (73.60,565.44)] font='AAAAAC+Arial-BoldMT' adv=0.389 text='r'>,
 <LTChar 77.103,563.532,79.605,572.532 matrix=[9.00,0.00,0.00,9.00, (77.10,5

In [17]:
def arrange_text(characters):
    """
    For each row find the characters in the row
    and sort them horizontally.
    """
    
    # find unique y0 (rows) for character assignment
    rows = sorted(list(set(c.bbox[1] for c in characters)), reverse=True)
    
    sorted_rows = []
    for row in rows:
        sorted_row = sorted([c for c in characters if c.bbox[1] == row], key=lambda c: c.bbox[0])
        sorted_rows.append(sorted_row)
    return sorted_rows

sorted_rows = arrange_text(raw_characters)
sorted_rows

[[<LTChar 666.240,581.504,671.576,589.504 matrix=[8.00,0.00,0.00,8.00, (666.24,583.20)] font='AAAAAB+ArialMT' adv=0.667 text='P'>,
  <LTChar 671.576,581.504,676.024,589.504 matrix=[8.00,0.00,0.00,8.00, (671.58,583.20)] font='AAAAAB+ArialMT' adv=0.556 text='a'>,
  <LTChar 676.024,581.504,680.472,589.504 matrix=[8.00,0.00,0.00,8.00, (676.02,583.20)] font='AAAAAB+ArialMT' adv=0.556 text='g'>,
  <LTChar 680.472,581.504,684.920,589.504 matrix=[8.00,0.00,0.00,8.00, (680.47,583.20)] font='AAAAAB+ArialMT' adv=0.556 text='e'>,
  <LTChar 704.880,581.504,709.328,589.504 matrix=[8.00,0.00,0.00,8.00, (704.88,583.20)] font='AAAAAB+ArialMT' adv=0.556 text='1'>,
  <LTChar 709.328,581.504,713.776,589.504 matrix=[8.00,0.00,0.00,8.00, (709.33,583.20)] font='AAAAAB+ArialMT' adv=0.556 text='0'>,
  <LTChar 713.776,581.504,716.000,589.504 matrix=[8.00,0.00,0.00,8.00, (713.78,583.20)] font='AAAAAB+ArialMT' adv=0.278 text=' '>,
  <LTChar 716.000,581.504,720.448,589.504 matrix=[8.00,0.00,0.00,8.00, (716.00,583.

In [18]:
col_margin = 0.5
def create_separators(sorted_rows, margin):
    """Creates bounding boxes to fill the space between columns"""
    separators = []
    for row in sorted_rows:
        for idx, c in enumerate(row[:-1]): 
            if (row[idx+1].bbox[0] - c.bbox[2]) > margin:
                bbox = (c.bbox[2], c.bbox[3], row[idx+1].bbox[0], row[idx+1].bbox[1])
                separator = pdfminer.layout.LTRect(linewidth=2, bbox=bbox)
                separators.append(separator)
    return separators

separators = create_separators(sorted_rows, col_margin)

In [20]:
def arrange_and_extract_text(characters, margin=0.5):
    
    rows = sorted(list(set(c.bbox[1] for c in characters)), reverse=True)
    
    row_texts = []
    for row in rows:
        
        sorted_row = sorted([c for c in characters if c.bbox[1] == row], key=lambda c: c.bbox[0])
        
        col_idx=0
        row_text = []
        for idx, char in enumerate(sorted_row[:-1]):
            if (sorted_row[idx+1].bbox[0] - char.bbox[2]) > margin:
                col_text = "".join([c.get_text() for c in sorted_row[col_idx:idx+1]])
                col_idx = idx+1
                row_text.append(col_text)
            elif idx==len(sorted_row)-2:
                col_text = "".join([c.get_text() for c in sorted_row[col_idx:]])
                row_text.append(col_text) 
        row_texts.append(row_text)
    return row_texts

text = arrange_and_extract_text(raw_characters)
text

[['Page', '10 of', '22'],
 ['TD Ameritrade Clearing, Inc.', 'Account', '454239302'],
 ['Proceeds from Broker and Barter Exchange Transactions'],
 ['OMB No. 1545-0715'],
 ['2020', '1099-B*   ', '02/04/2021'],
 ['(continued)'],
 ['(Lines 2 & 5) '],
 ['SHORT TERM TRANSACTIONS FOR COVERED TAX LOTS [Ordinary gains or losses are identified in the Additional information column] '],
 ['Report on Form 8949, Part I with Box A checked. Basis is provided to the IRS. (Line 12)'],
 ['“Gain or loss (-)” is NOT reported to the IRS.'],
 ['1a-', 'Description', 'of', 'property/CUSIP/Symbol'],
 ['1c-',
  'Date',
  '1d-',
  'Proceeds',
  '&',
  '1f-',
  'Accrued',
  'mkt',
  'disc',
  '(D)',
  '&',
  'Gain',
  'or',
  'loss(-)'],
 ['sold',
  'or',
  '6-',
  'Reported',
  '1b-',
  'Date',
  '1e-',
  'Cost',
  'or',
  '1g-',
  'Wash',
  'sale',
  'loss',
  '7-',
  'Loss',
  'not',
  'allowed',
  '(X)'],
 ['disposed',
  'Quantity',
  '(G)ross',
  'or',
  '(N)et',
  'acquired',
  'other',
  'basis',
  'disallo

In [26]:

def generate_current_page_dataframe(text):
    columns = ['Date sold or disposed', 'Quantity', 'Proceeds', 'Date acquired', 'Cost or other basis', 'Wash sale loss disallowed (W)', 'Code', 'Gain or loss(-)', 'Additional information']
    df = pd.DataFrame(columns=columns)
    for row in text:
        if len(row) == 5:
            row = row.copy()
            row.insert(0, "-")
            row.insert(1, "-")
            row.insert(3, "-")
            row.insert(8, "-")
            temp = pd.DataFrame([row], columns=columns)
            df = df.append(temp, ignore_index=True)
        elif len(row) == 9:
            df_row = pd.DataFrame([row], columns=columns)
            df = df.append(df_row, ignore_index=True)
        elif len(row) == 8 and row[5] == "W":
            row = row.copy()
            row.insert(3, "Various")
            temp = pd.DataFrame([row], columns=columns)
            df = df.append(temp, ignore_index=True)
        else:
            continue
    return df

In [27]:
df = generate_current_page_dataframe(text)

In [28]:
df

,Date sold or disposed,Quantity,Proceeds,Date acquired,Cost or other basis,Wash sale loss disallowed (W),Code,Gain or loss(-),Additional information
0,08/17/20,10.000,993.30,Various,"1,161.32",168.02,W,0.00,Total of 2 transactions
1,08/18/20,4.000,121.32,08/11/20,437.82,316.50,W,0.00,Option sale
2,-,-,"1,257.93",-,"2,748.51",484.52,W,"-1,006.06",-
3,08/11/20,3.000,342.98,08/11/20,502.00,159.02,W,0.00,Option sale
4,08/17/20,2.000,318.66,08/11/20,434.01,115.35,W,0.00,Option sale
5,-,-,"1,078.97",-,"1,683.36",274.37,W,-330.02,-
